# Whiting I
This notebook explores a very simple analysis to show the utility of notebooks in general, and some of the features of the JupyterLab environment specifically.

This notebook uses data from DR14 of the SDSS survey using the globular cluster Whiting I as a starting point.  See the [finding chart](http://skyserver.sdss.org/dr8/en/tools/chart/navi.asp?ra=30.7367&dec=-3.25319&opt=).

## Before running the notebook, get the data

### Make a place to keep the data in your env.
```
$> mkdir $HOME/data
$> cd $HOME/data
```

### Get the catalog.
```
$> curl https://lsst-web.ncsa.illinois.edu/~krughoff/data/MyResult_20171010_5.csv -o cluster.csv
```

This catalog was generated using [CasJobs](https://skyserver.sdss.org/CasJobs/SubmitJob.aspx) using the following query
```sql
SELECT si.ra, si.dec, si.type, si.psfMag_u, si.psfMag_g, si.psfMag_r, si.psfMag_i, si.psfMag_z,
  si.psfMagErr_u, si.psfMagErr_g, si.psfMagErr_r, si.psfMagErr_i, si.psfMagErr_z,
  si.modelMag_u, si.modelMag_g, si.modelMag_r, si.modelMag_i, si.modelMag_z,
  si.modelMagErr_u, si.modelMagErr_g, si.modelMagErr_r, si.modelMagErr_i, si.modelMagErr_z
FROM photoprimary si WHERE
si.ra between 30.7375-0.1 AND 30.7375+0.1 AND
si.dec between -3.2528-0.1 AND -3.2528+0.1
```
### Get an image

This image was identified by using the SDSS SIAP service.  Specifically by looking through the response from [this URL](http://skyserver.sdss.org/dr14/SkyServerWS/SIAP/getSIAP?POS=30.7375,-3.2528&SIZE=0.001&FORMAT=image/fits)

```
$> curl -LO http://dr14.sdss.org/sas/dr14/eboss/photoObj/frames/301/7778/1/frame-r-007778-1-0522.fits.bz2
$> bunzip2 frame-r-007778-1-0522.fits.bz2
```

## Simple density analysis

Initial imports

In [1]:
import numpy, os, boto # this will fail on systems that don't have boto
from matplotlib import pylab as plt
plt.rcParams['figure.dpi'] = 120
from scipy.ndimage.filters import gaussian_filter

import astropy.io.fits as fits
from astropy.wcs import WCS

ModuleNotFoundError: No module named 'boto'

Set some useful variables

In [ ]:
ra = 30.7375
dec = -3.2528
radius = 1.2/60

Read in the catalog

In [ ]:
home = os.getenv('HOME')
cat = numpy.genfromtxt(os.path.join(home,"data/cluster.csv"), names=True, dtype=None, delimiter=',')

Does the catalog look reasonable?

In [ ]:
def plot(x, y):
    plt.scatter(x, y, alpha=0.3, s=10, c='black')
    plt.xlabel('RA (deg)')
    plt.ylabel('Declination (deg)')
    plt.show()

plot(cat['ra'], cat['dec'])

SDSS gives us a star galaxy separation.  Let's use it!

In [ ]:
stars = cat[numpy.where(cat['type']==6)[0]]
galaxies = cat[numpy.where(cat['type']==3)[0]]

Stars are clustered as expected

In [ ]:
plot(stars['ra'], stars['dec'])

Galaxies, not as much.

In [ ]:
plot(galaxies['ra'], galaxies['dec'])

Select cluster members spatially.

In [ ]:
cluster_stars = stars[numpy.where(numpy.hypot(stars['ra']-ra, stars['dec']-dec) < radius)[0]]

Compare spatially selected stars with all stars in a color-magnitude diagram.

In [ ]:
plt.scatter(stars['psfMag_r']-stars['psfMag_i'], stars['psfMag_r'], alpha=0.3, s=10, c='black')
plt.scatter(cluster_stars['psfMag_r']-cluster_stars['psfMag_i'], cluster_stars['psfMag_r'], c='red', alpha=0.3, s=10)
plt.ylim(25, 13)
plt.xlim(-1, 3.)
plt.xlabel('r-i')
plt.ylabel('r')
plt.show()

Make a reasonable cut in `r-i` color space.

In [ ]:
rmi = stars['psfMag_r']-stars['psfMag_i']
cluster = stars[numpy.where((rmi>-0.25) & (rmi<0.5))[0]]

Now we can plot the color selected cluster members

In [ ]:
plot(cluster['ra'], cluster['dec'])

Histogram the star counts and smooth on a reasonable scale.

In [ ]:
H, xedges, yedges = plt.histogram2d(cluster['ra'], cluster['dec'], range=[[30.6,30.85], [-3.36, -3.15]], bins=(30,30))
levels = (0.1, 0.4, 0.7, 1.0, 1.3, 1.6, 1.8)
smoothed_hist = gaussian_filter(H, 3.0) # three bin sigma

Compute the extent of the subimage so we can plot the contours.

In [ ]:
pixmin = (0, 250)
pixmax = (600, 850)
im = fits.open(os.path.join(home,'data/frame-r-007778-1-0522.fits'))
wcs = WCS(im[0].header)
extent = wcs.wcs_pix2world([pixmin[::-1], pixmax[::-1]], 0)

Plot image and contours.  Note transposed axes.

In [ ]:
plt.contour(0.5*(yedges[1:] + yedges[:-1]), 0.5*(xedges[1:] + xedges[:-1]), smoothed_hist, levels)
plt.imshow(im[0].data[pixmin[0]:pixmax[0], pixmin[1]:pixmax[1]], vmin=-0.01, vmax=0.2, cmap='gray',
           extent=(extent[0][1], extent[1][1], extent[1][0], extent[0][0]))
plt.xlabel("Declination (deg)")
plt.ylabel("RA (deg)")
plt.show()